In [2]:
import tqdm
import urllib
import os.path
import sqlite3
import pandas as pd
import re
import numpy as np
import threading
from w3lib.url import safe_url_string
from urllib.parse import urlparse, urlunparse
from bs4 import BeautifulSoup as bf


AMAZON_URL = "https://www.amazon.com/s?k={}&i=movies-tv"
AMAZON_URL_HEAD = "https://www.amazon.com"
IMDB_URL =  "https://www.imdb.com/title/tt{}/?ref_=fn_al_tt_1"

MOVIELENS_PATH = './'
OUTPUT_CSV = "./prices.csv"
DATABASE_PATH = "./database.db"
REQUEST_TIMEOUT = 5.0

### Load movies.csv and links.csv

In [3]:
movies_df = pd.read_csv(MOVIELENS_PATH + 'movies.csv', dtype=str)
print(movies_df.head())

links_df = pd.read_csv(MOVIELENS_PATH + 'links.csv',dtype=str)
print(links_df.head())

  movieId                               title  \
0       1                    Toy Story (1995)   
1       2                      Jumanji (1995)   
2       3             Grumpier Old Men (1995)   
3       4            Waiting to Exhale (1995)   
4       5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
  movieId   imdbId tmdbId
0       1  0114709    862
1       2  0113497   8844
2       3  0113228  15602
3       4  0114885  31357
4       5  0113041  11862


### Join movies_df and links_df by movieId

In [4]:
merged_df = pd.merge(movies_df, links_df, on='movieId')
print(merged_df.head())

  movieId                               title  \
0       1                    Toy Story (1995)   
1       2                      Jumanji (1995)   
2       3             Grumpier Old Men (1995)   
3       4            Waiting to Exhale (1995)   
4       5  Father of the Bride Part II (1995)   

                                        genres   imdbId tmdbId  
0  Adventure|Animation|Children|Comedy|Fantasy  0114709    862  
1                   Adventure|Children|Fantasy  0113497   8844  
2                               Comedy|Romance  0113228  15602  
3                         Comedy|Drama|Romance  0114885  31357  
4                                       Comedy  0113041  11862  


### Create Database and Table

Table structure: movieId (int)，imdbId (varchar30), title (varchar50), pageUrl (varchar300), price (double)

In [5]:
database = sqlite3.connect(DATABASE_PATH, check_same_thread=False)
cursor = database.cursor()
cursor.execute('create table if not exists prices (movieId integer primary key, imdbId varchar(30) NOT NULL, title varchar(50) NOT NULL, pageUrl varchar(300), price double)')
database.commit()

### Query exist tables

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Current table: ", cursor.fetchall())

Current table:  [('prices',)]


### Query table structure

In [6]:
cursor.execute("PRAGMA table_info(prices)")
print("Table prices: ", cursor.fetchall())

Table prices:  [(0, 'movieId', 'integer', 0, None, 1), (1, 'imdbId', 'varchar(30)', 1, None, 0), (2, 'title', 'varchar(50)', 1, None, 0), (3, 'pageUrl', 'varchar(300)', 0, None, 0), (4, 'price', 'double', 0, None, 0)]


### Insert movie data if not exist

In [7]:
row_num = merged_df.shape[0]
pbar = tqdm.trange(row_num, position=0, leave=True)
for i in pbar:
    pbar.set_description("Processing row %d" % i)
    row_item = merged_df.iloc[i]
    cursor.execute("SELECT * FROM prices WHERE movieId = ?;", (int(row_item['movieId']),))
    if len(cursor.fetchall()) == 0:
        cursor.execute("INSERT INTO prices (movieId, imdbId, title, pageUrl, price) VALUES (?, ?, ?, NULL, NULL)",
                       (int(row_item['movieId']), row_item['imdbId'], row_item['title']))
        database.commit()

Processing row 27277: 100%|██████████| 27278/27278 [04:47<00:00, 95.04it/s]


### Display first 5 data from database

In [7]:
cursor.execute("SELECT * FROM prices LIMIT 0, 10;")
print(cursor.fetchall())

[(1, '0114709', 'Toy Story (1995)', 'https://www.amazon.com/Toy-Story-Tim-Allen/dp/B0094KTAEY/ref=sr_1_1?dchild=1&keywords=Toy+Story+1995&qid=1619252106&s=movies-tv&sr=1-1', None), (2, '0113497', 'Jumanji (1995)', 'https://www.amazon.com/Jumanji-Robin-Williams/dp/B00BZBQ4GW/ref=sr_1_1?dchild=1&keywords=Jumanji+1995&qid=1619252109&s=movies-tv&sr=1-1', None), (3, '0113228', 'Grumpier Old Men (1995)', 'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619252112&s=movies-tv&sr=1-1', None), (4, '0114885', 'Waiting to Exhale (1995)', 'https://www.amazon.com/Celebrated-Women-Color-Film-Waiting/dp/B003RACH3I/ref=sr_1_1?dchild=1&keywords=Waiting+to+Exhale+1995&qid=1619252113&s=movies-tv&sr=1-1', None), (5, '0113041', 'Father of the Bride Part II (1995)', 'https://www.amazon.com/Father-Bride-Part-Theatrical-Version/dp/B072R146S8/ref=sr_1_1?dchild=1&keywords=Father+of+the+Bride+Part+II+1995&qid=1619252114&s=movies-tv&sr=1-1',

In [6]:
def getPageUrl(title):
    url = AMAZON_URL.format(title.replace(" ", "+"))
#     url = safe_url_string(url, encoding="utf-8")
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def getPageUrlFromIMDB(imdbId):
    url = AMAZON_URL.format(title.replace(" ", "+"))
    req = urllib.request.Request(url)
    req.add_header('User-Agent', getRandomUserAgent())
    html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
    obj = bf(html.read(),'html.parser')
    firstItem = obj.body.find_all('div',class_="a-section a-spacing-medium")[0]
    aElement = firstItem.find_all('a', class_="a-link-normal a-text-normal")[0]
    pageUrl = AMAZON_URL_HEAD + aElement['href']
    return pageUrl

def removeBrackets(title):
    return title[:-7] + " " + title[-5:-1]

def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

def iriToUri(iri):
    parts= urlparse(iri)
    return urlunparse(
        part.encode('idna') if parti==1 else urlEncodeNonAscii(part.encode('utf-8'))
        for parti, part in enumerate(parts)
    )

def runGetPage(movie_list):
    pbar = tqdm.tqdm(range(len(movie_list)), position=0, leave=True)
    for i in pbar:
        try:
            pbar.set_description("Processing row %d" % i)
            m = movie_list[i]
            movie_id = m[0]
            movie_title = m[2]
            page_url = m[3]
            if page_url is None:
                page_url = getPageUrl(removeBrackets(movie_title))
                updateDataBase(page_url, movie_id)
        except KeyboardInterrupt:
            sys.exit()
            pass
        except Exception as e:
            print("Exception at row %d" % i)
            print(e)
            continue
            
def getRandomUserAgent():
    import requests
    import requests_random_user_agent

    s = requests.Session()
    return s.headers['User-Agent']

def list_split(items, n):
    return [items[i:i+n] for i in range(0, len(items), n)]

lock = threading.Lock()

def updateDataBase(page_url, movie_id):
    try:
        lock.acquire(True)
        cursor.execute("UPDATE prices SET pageUrl = ? WHERE movieId = ?;", (page_url, movie_id))
        database.commit()
    finally:
        lock.release()

In [99]:
cursor.execute("UPDATE prices SET pageUrl = NULL WHERE imdbId = ?;", ("0112682", ))
database.commit()

In [9]:
getPageUrl(removeBrackets("Grumpier Old Men (1995)"))

'https://www.amazon.com/Grumpier-Old-Men-Jack-Lemmon/dp/B00AUOUG8W/ref=sr_1_1?dchild=1&keywords=Grumpier+Old+Men+1995&qid=1619257117&s=movies-tv&sr=1-1'

In [122]:
imdbId = "0114709"
url = IMDB_URL.format(imdbId)
req = urllib.request.Request(url)
req.add_header('User-Agent', getRandomUserAgent())
html = urllib.request.urlopen(req, timeout=REQUEST_TIMEOUT)
obj = bf(html.read(),'html.parser')
print(obj.body.find_all('button',class_="buybox__button"))

[]


### Load data from database to Dataframe

In [ ]:
output_df = pd.DataFrame(columns=['movieId', 'title', 'pageUrl', 'price'])
cursor.execute("SELECT * FROM prices;")
pbar = tqdm.tqdm(cursor.fetchall(), position=0, leave=True)
for row in pbar:
    pbar.set_description("Processing %d" % 1)
    row_dict = {'movieId': row[0], 'title': row[1], 'pageUrl': row[2], 'price': row[3]}
    output_df = output_df.append(row_dict, ignore_index=True)

Processing 1:  77%|███████▋  | 20920/27278 [01:58<00:37, 170.03it/s]

### Write into CSV file

In [121]:
output_df.to_csv(OUTPUT_CSV, index=False)

In [116]:
getPageUrl('Jumanji 1995')

'https://www.amazon.com/Jumanji-Robin-Williams/dp/B00BZBQ4GW/ref=sr_1_1?dchild=1&keywords=Jumanji+1995&qid=1619255969&s=movies-tv&sr=1-1'

In [62]:
!pip3 install urlparse

ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [92]:
from w3lib.url import safe_url_string
url = safe_url_string('http://example.org/Ñöñ-ÅŞÇİİ/', encoding="utf-8")
print(url)

http://example.org/%C3%91%C3%B6%C3%B1-%C3%85%C5%9E%C3%87%C4%B0%C4%B0/


In [106]:
!pip install requests-random-user-agent

     |████████████████████████████████| 47 kB 1.6 MB/s eta 0:00:01


In [7]:
import threading
import requests
from lxml import etree
from urllib import request
import os
import re

class Consumer(threading.Thread):
    def __init__(self, l, *args, **kwargs):
        super(Consumer, self).__init__(*args,**kwargs)
        self.l = l

    def run(self):
        runGetPage(self.l)

In [8]:
cursor.execute("SELECT * FROM prices;")
movie_list = cursor.fetchall()
WORKERS_NUM = 4
split_list = np.array_split(movie_list, 5)

In [9]:
for i in range(WORKERS_NUM):
    t = Consumer(split_list[i])
    t.start()

Processing row 83:   1%|          | 32/5456 [00:00<00:33, 160.36it/s]

Exception at row 8
'ascii' codec can't encode character '\xed' in position 23: ordinal not in range(128)
Exception at row 10
'ascii' codec can't encode character '\xe8' in position 33: ordinal not in range(128)
Exception at row 13
'ascii' codec can't encode character '\xe8' in position 40: ordinal not in range(128)
Exception at row 19
'ascii' codec can't encode character '\xda' in position 29: ordinal not in range(128)
Exception at row 20
'ascii' codec can't encode character '\xe1' in position 29: ordinal not in range(128)
Exception at row 21
'ascii' codec can't encode character '\xe4' in position 20: ordinal not in range(128)
Exception at row 28
'ascii' codec can't encode character '\xe9' in position 40: ordinal not in range(128)
Exception at row 27
'ascii' codec can't encode character '\xfb' in position 40: ordinal not in range(128)


Processing row 389:   5%|▌         | 282/5456 [00:00<00:17, 288.05it/s]

Exception at row 113
'ascii' codec can't encode character '\xe9' in position 58: ordinal not in range(128)
Exception at row 281
'ascii' codec can't encode character '\xfb' in position 25: ordinal not in range(128)
Exception at row 290
'ascii' codec can't encode character '\xe9' in position 10: ordinal not in range(128)


Processing row 473:   7%|▋         | 407/5456 [00:00<00:13, 374.33it/s]

...

Processing row 2479:  45%|████▌     | 2479/5455 [1:02:29<1:00:05,  1.21s/it]

Exception at row 2423
'ascii' codec can't encode character '\xe8' in position 30: ordinal not in range(128)


Processing row 2427:  44%|████▍     | 2426/5456 [1:02:34<1:43:54,  2.06s/it]

Exception at row 2426
'ascii' codec can't encode character '\xe9' in position 45: ordinal not in range(128)


Processing row 2484:  46%|████▌     | 2484/5455 [1:02:36<1:13:04,  1.48s/it]

Processing row 2512:  46%|████▌     | 2512/5455 [1:03:18<1:10:22,  1.43s/it]

Exception at row 2511
list index out of range


Processing row 3246:  60%|█████▉    | 3247/5456 [1:03:24<1:01:48,  1.68s/it]

Exception at row 2515
'ascii' codec can't encode character '\xe9' in position 12: ordinal not in range(128)


Processing row 2460:  45%|████▌     | 2460/5456 [1:03:32<1:24:04,  1.68s/it]

...

Exception at row 5443
list index out of range


Processing row 5451: 100%|█████████▉| 5451/5456 [1:59:09<00:06,  1.23s/it]

Exception at row 5450
list index out of range


Processing row 5455: 100%|██████████| 5456/5456 [1:59:17<00:00,  1.31s/it]

Exception at row 5454
list index out of range
Exception at row 5455
'ascii' codec can't encode character '\xe1' in position 51: ordinal not in range(128)
